# Movie Review ML

## Importações 

### Bibliotecas para cálculos com array 
Pacotes de biblioteca cientificas para calculos com arrays n-dimensionais e outras funções

In [ ]:
import os
import pandas
import numpy

### Modelos

Modelos de classificação linear, Naive Bayes e SVN como soluções mais utilizadas nas pesquisas que são simples e que atingem o objetivo de lidar com textos.

In [ ]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier

### Engenharia de características

Foram utilizadas as bibliotecas TextBlob e Tfidf, sendo essa propria da Scikit learn 

In [ ]:
from dense_transform import DenseTransformer, tokenizer, tokenizer_correct
from sklearn.feature_extraction.text import TfidfTransformer,  CountVectorizer, TfidfVectorizer

## Gráficos

Para a geração de gráficos foi utilizado o Matplot lib

In [ ]:
from matplotlib import pyplot

### Validação de Modelo

Foi utilizada a biblioteca sklearn

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

### Pipelines

Processamento dos dados utilizando filtragem pelos metodos: gaussian, bernoulli, multinomial, Linear e SGDClassifier

In [ ]:
from sklearn.pipeline import Pipeline

### Time

Biblioteca para acesso ao tempo e conversões

In [ ]:
import time

### Obtenção de Data

Método utilizado para a obtenção dos dados que irão ser tratados

In [ ]:
def obtain_data():
    t0 = time.time()
    global paths_df
    # Load dirs name
    cur_dir = os.path.realpath('.')
    pos_dir = os.path.join(cur_dir, 'train-pos')
    neg_dir = os.path.join(cur_dir, 'train-neg')


    # Load files names
    list_pos_dir = [ (os.path.join(pos_dir, x), 1) for x in os.listdir(pos_dir)][:11]
    list_neg_dir = [ (os.path.join(neg_dir, x), 0) for x in os.listdir(neg_dir)][:11]
    print("registers: {}".format(len(list_pos_dir+list_neg_dir)))
    print("Attention with 6000 registers it will consume about 5+GB of ram")
    # input("Continue? or press CTRL+C")

    # Mount data with label data frame
    paths_df = pandas.DataFrame(list_pos_dir+list_neg_dir, columns=['path', 'label'])

    print("It took: " + str(time.time() - t0) + " to obtain data")

### Mostrar números de tokens

Metodo utilizado para a obtenção do número de tokens da aplicação

In [ ]:
def show_number_of_tokens():
    t0 = time.time()
    # Verify difference between size of tokens with tokenizer stem, stopwords
    tfidf_stem = TfidfVectorizer(input='filename', stop_words='english', tokenizer=tokenizer)
    tfidf_tokenizer_correct = TfidfVectorizer(input='filename', stop_words='english', tokenizer=tokenizer_correct)
    tfidf_stem.fit(paths_df.path.values)
    tfidf_stem.get_feature_names()
    tfidf_stop = TfidfVectorizer(input='filename', stop_words='english')
    tfidf_word = TfidfVectorizer(input='filename')


    # Simple benchmark for number of features
    result = []
    for tfidf in [tfidf_stem, tfidf_word, tfidf_stop, tfidf_tokenizer_correct]:
        tfidf.fit(paths_df.path.values)
        result.append(len(tfidf.get_feature_names()))

    result = pandas.DataFrame(result, columns=['len_of_features'], index=['tfidf_stem', 'tfidf_word', 'tfidf_stop', 'tfidf_tokenizer_correct'])
    result = result.assign(difference=lambda x: (x.len_of_features - x.len_of_features.min()))
    print(result)
    pyplot.figure(1)
    pyplot.bar([1,2,3,4], result.difference.values)
    pyplot.xticks([1,2,3], result.index.values)
    pyplot.ylabel('Number of tokens')
    pyplot.xlabel('Method of tf-idf')
    print("It took: " + str(time.time() - t0) + " to show number of tokens")

### Criação de Pipelines

Método para criação de pipelines para tratamento dos dados em etapas utilizando os metodos de gaussian, bernoulli, multinomial, Linear e SGDClassifier

In [ ]:
def create_pipes():
    global pipes
    t0 = time.time()
    # Create pipes
    pipes = {
        'gaussianNB': Pipeline([
          ('vect', TfidfVectorizer(input='filename')),
          ('dense', DenseTransformer()),
          ('gnb', GaussianNB())
        ]),
        'bernoulliNB': Pipeline([
          ('vect', TfidfVectorizer(input='filename', binary=True)),
          ('dense', DenseTransformer()),
          ('gnb', BernoulliNB())
        ]),
        'multinomialNB': Pipeline([
          ('vect', TfidfVectorizer(input='filename')),
          ( 'gnb', MultinomialNB())
        ]),
        'linearSVC': Pipeline([
          ('vect', TfidfVectorizer(input='filename')),
          ( 'gnb', LinearSVC())
        ]),
        'sgdclassifier': Pipeline([
          ('vect', TfidfVectorizer(input='filename')),
          ( 'gnb', SGDClassifier(max_iter=5))
        ]),
    }
    print("It took: " + str(time.time() - t0) + " to create pipes")
# Method to return params from pipe params adjusts